# Hybrid
* Movie title and user as input
* Use content based to compute 25 most similar movies
* Compute the predicted ratings using a collaborative model
Return top 10

In [2]:
import pandas as pd
import numpy as np

In [5]:
# 100000 ratings 9000movies 700 users
ratings = pd.read_csv('../data/ratings_small.csv')
ratings.head()


,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [4]:
r_matrix = ratings.pivot_table(values='rating',index ='movieId', columns ='userId')
r_matrix.head()

userId,1,2,3,4,5,6,7,8,9,10,...,662,663,664,665,666,667,668,669,670,671
movieId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,4.0,NaN,...,NaN,4.0,3.5,NaN,NaN,NaN,NaN,NaN,4.0,5.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.0,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
r_matrix.shape

(9066, 671)

In [8]:
r_matrix_dummy =r_matrix.copy().fillna(0)

In [9]:
r_matrix_dummy.head()

userId,1,2,3,4,5,6,7,8,9,10,...,662,663,664,665,666,667,668,669,670,671
movieId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,4.0,0.0,...,0.0,4.0,3.5,0.0,0.0,0.0,0.0,0.0,4.0,5.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
from sklearn.metrics.pairwise import cosine_similarity

In [11]:
cosine_sim = cosine_similarity(r_matrix_dummy, r_matrix_dummy)

In [14]:
type(cosine_sim)

pandas.core.frame.DataFrame

In [13]:
cosine_sim = pd.DataFrame(cosine_sim, index =r_matrix.index,columns =r_matrix.index)

In [16]:
movies_list = pd.read_csv('../data/movie_ids.csv')
movies_list.shape

(9219, 3)

In [18]:
movies_list.head()

,title,movieId,id
0,Toy Story,1,862.0
1,Jumanji,2,8844.0
2,Grumpier Old Men,3,15602.0
3,Waiting to Exhale,4,31357.0
4,Father of the Bride Part II,5,11862.0


In [66]:
movienamepicker = movies_list.set_index('movieId')
movienamepicker.head()

,title,id
movieId,,
1,Toy Story,862.0
2,Jumanji,8844.0
3,Grumpier Old Men,15602.0
4,Waiting to Exhale,31357.0
5,Father of the Bride Part II,11862.0


In [37]:
#cosine_sim_map =pd.Series(movies_list.index, index=movies_list['title'])
cosine_sim_map = movies_list.set_index('title')
#del cosine_sim_map.index.name

cosine_sim_map.head()


,movieId,id
title,,
Toy Story,1,862.0
Jumanji,2,8844.0
Grumpier Old Men,3,15602.0
Waiting to Exhale,4,31357.0
Father of the Bride Part II,5,11862.0


In [44]:
#Fucnction tester(idx)
idx =cosine_sim_map.loc['Grumpier Old Men','movieId']
print(idx)

3.0


In [57]:
# Takes title and returns idx
def idx(title):
    idx =cosine_sim_map.loc[title,'movieId']

    return idx
x = idx('Grumpier Old Men')
print(x)
    

3.0


In [63]:
# Takes title and returns to 25 sorted sim_scores
def sim_scores(title):
    idx =cosine_sim_map.loc[title,'movieId']
    #works sim_scores = list(enumerate(list(cosine_sim.iloc[int(idx)])))
    sim_scores = list(enumerate(cosine_sim.iloc[int(idx)]))
    sim_scores = sorted(sim_scores, key =lambda x: x[1], reverse = True)
    sim_scores = sim_scores[1:26]
    
    return sim_scores
x = sim_scores('Grumpier Old Men')
print(x)
    

[(7, 0.5015126473794767), (379, 0.3956868046034364), (462, 0.3872983346207417), (2644, 0.36464571100446486), (241, 0.35677877654950424), (279, 0.32893691312298384), (177, 0.3263569759322527), (215, 0.3263569759322527), (255, 0.3263569759322527), (308, 0.3263569759322527), (424, 0.3263569759322527), (595, 0.3263569759322527), (784, 0.3263569759322527), (2839, 0.3263569759322527), (3271, 0.3263569759322527), (3287, 0.3263569759322527), (3330, 0.3263569759322527), (3977, 0.3263569759322527), (4223, 0.3263569759322527), (4260, 0.3263569759322527), (83, 0.32337568953426626), (3451, 0.31661277257078557), (2713, 0.30960941226920163), (509, 0.30581037989957505), (3719, 0.30581037989957505)]


In [67]:
# Takes title and returns to 25 sorted movie names
def movienames(title):
    idx =cosine_sim_map.loc[title,'movieId']
    #works sim_scores = list(enumerate(list(cosine_sim.iloc[int(idx)])))
    sim_scores = list(enumerate(cosine_sim.iloc[int(idx)]))
    sim_scores = sorted(sim_scores, key =lambda x: x[1], reverse = True)
    sim_scores = sim_scores[1:26]
    movieids = [i[0] for i in sim_scores]
    movienames = movienamepicker.iloc[movieids][['title']]   
    
    
    return movienames
x = movienames('Grumpier Old Men')
print(x)
    

                            title
movieId                          
8                    Tom and Huck
429                     Cabin Boy
516               Renaissance Man
3272               Bad Lieutenant
269                  Mi Vida Loca
313             The Swan Princess
201                  Three Wishes
243                         Gordy
283              New Jersey Drive
343         The Baby-Sitters Club
476                   The Inkwell
694                The Substitute
953         It's a Wonderful Life
3535              American Psycho
4066      I'm Gonna Git You Sucka
4090     The Brave Little Toaster
4158                   Monkeybone
5117                   Funny Lady
5556                   FearDotCom
5621                   The Tuxedo
89                   Nick of Time
4326          Mississippi Burning
3380                  Railroaded!
569             Little Big League
4719                Osmosis Jones


In [73]:
# Takes title and returns to 25 sorted movie names
def movienames(userId,title):
    idx =cosine_sim_map.loc[title,'movieId']
    #works sim_scores = list(enumerate(list(cosine_sim.iloc[int(idx)])))
    sim_scores = list(enumerate(cosine_sim.iloc[int(idx)]))
    sim_scores = sorted(sim_scores, key =lambda x: x[1], reverse = True)
    sim_scores = sim_scores[1:26]
    movieids = [i[0] for i in sim_scores]
    movienames = movienamepicker.iloc[movieids][['title']]   
    
    movienames['est'] = movienames.apply(lambda x: svd.predict(userId, 'movieId').est)
    
    return movienames
x = movienames(1,'Grumpier Old Men')
print(x)
    


                            title  est
movieId                               
8                    Tom and Huck  NaN
429                     Cabin Boy  NaN
516               Renaissance Man  NaN
3272               Bad Lieutenant  NaN
269                  Mi Vida Loca  NaN
313             The Swan Princess  NaN
201                  Three Wishes  NaN
243                         Gordy  NaN
283              New Jersey Drive  NaN
343         The Baby-Sitters Club  NaN
476                   The Inkwell  NaN
694                The Substitute  NaN
953         It's a Wonderful Life  NaN
3535              American Psycho  NaN
4066      I'm Gonna Git You Sucka  NaN
4090     The Brave Little Toaster  NaN
4158                   Monkeybone  NaN
5117                   Funny Lady  NaN
5556                   FearDotCom  NaN
5621                   The Tuxedo  NaN
89                   Nick of Time  NaN
4326          Mississippi Burning  NaN
3380                  Railroaded!  NaN
569             Little Bi

# Prep done use SVD

In [70]:
from surprise import SVD, Reader, Dataset, evaluate

reader = Reader()
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader) #need evaluation
data.split(n_folds=5)
svd = SVD()
trainset =data.build_full_trainset()
svd.train(trainset)

In [26]:
evaluate(svd, data, measures =['RMSE'])

/home/scrybe/.local/lib/python3.6/site-packages/surprise/evaluate.py:66: UserWarning: The evaluate() method is deprecated. Please use model_selection.cross_validate() instead.
  'model_selection.cross_validate() instead.', UserWarning)
/home/scrybe/.local/lib/python3.6/site-packages/surprise/dataset.py:193: UserWarning: Using data.split() or using load_from_folds() without using a CV iterator is now deprecated. 
  UserWarning)


Evaluating RMSE of algorithm SVD.

------------
Fold 1
RMSE: 0.9069
------------
Fold 2
RMSE: 0.8936
------------
Fold 3
RMSE: 0.8946
------------
Fold 4
RMSE: 0.8992
------------
Fold 5
RMSE: 0.8864
------------
------------
Mean RMSE: 0.8961
------------
------------


CaseInsensitiveDefaultDict(list,
                           {'rmse': [0.9069180166963678,
                             0.893552569146935,
                             0.8946113247220904,
                             0.8992397085709837,
                             0.8864173691239492]})

In [ ]:
#mappings
id_to_title = movies_list.set_index('id')
#id_to_title = movies_list.set_index('movieId')
title_to_id = movies_list.set_index('title')

In [ ]:
print(title_to_id)

In [ ]:
def hybrid(userId,title):
    idx =cosine_sim_map.loc[title.'movieId']
    sim_scores = list(enumerate(list(cosine_sim.iloc[idx])))

    sim_scores = sorted(sim_scores, key =lambda x: x[1], reverse = True)
    sim_scores = sim_scores[1:26]
    movieids = [i[0] for i in sim_scores]
    movies_meta = movies_list.iloc[movieids][['title']]   
    movies_meta['est'] = movies_meta.apply(lambda x: svd.predict(userId, id_to_title.loc[x]['movieId']).est, axis =0)
    #movies_meta['est'] = movies_meta['id'].apply(lambda x: svd.predict(userId, id_to_title.loc[x]['movieId']).est)
    movies_meta =movies_meta.sort_values('est', ascending=False)
    
    return movies_meta.head(10)

In [28]:
idx =cosine_sim_map['Jumanji']
sim_scores = list(enumerate(list(cosine_sim.iloc[idx])))

sim_scores = sorted(sim_scores, key =lambda x: x[1], reverse = True)
sim_scores = sim_scores[1:26]
movieids = [i[0] for i in sim_scores]
movies_meta = movies_list.iloc[movieids][['title']]
#new_index = movies_meta.set_index('title')
#movies_meta.index.name = 'id'
#movies_meta['est'] = movies_meta['index'].apply(lambda x: svd.predict(userId, id_to_title.loc[x]['movieId']).est)
# movies_meta =movies_meta.sort_values('est', ascending=False)
#print(movieids)
movies_meta.head()

KeyError: 'Jumanji'

In [ ]:
userId =1
movies_meta['est'] = movies_meta['title'].apply(lambda x: svd.predict(userId, title_to_id.loc[x]['movieId']).est)

In [ ]:
title_to_id.loc['The Lion King']['movieId']

In [ ]:
movies_meta['title']

In [ ]:
hybrid(1,'Jumanji')